# Билет 21

# Задание 1

# Задание 2

<img src="img/21_2.png" width=800>

In [76]:
import pandas as pd

In [77]:
df: pd.DataFrame = pd.read_csv(
    'data/task_2/sp500hst.txt',
    sep=',',
    header=None,
    names=['date', 'ticker', *[f'what_is_{i}' for i in range(4)], 'vol']
)
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')
df.head()

,date,ticker,what_is_0,what_is_1,what_is_2,what_is_3,vol
0,2009-08-21,A,25.60,25.6100,25.220,25.55,34758
1,2009-08-24,A,25.64,25.7400,25.330,25.50,22247
2,2009-08-25,A,25.50,25.7000,25.225,25.34,30891
3,2009-08-26,A,25.32,25.6425,25.145,25.48,33334
4,2009-08-27,A,25.50,25.5700,25.230,25.54,70176


In [78]:
pivot_table = df.pivot_table(
    'vol',
    index=df['date'].rename(None),
    columns=df['ticker'].rename(None),
)
pivot_table.to_csv('out/21_2.csv')
pivot_table.head()

,A,AA,AAPL,ABC,ABT,ACE,ACS,ADBE,ADI,ADM,...,XL,XLNX,XOM,XRAY,XRX,XTO,YHOO,YUM,ZION,ZMH
2009-08-21,34758.0,338295.0,148597.0,33179.0,84699.0,NaN,17241.0,51086.0,64696.0,40472.0,...,85971.0,71158.0,261666.0,9849.0,42005.0,55384.0,235459.0,31962.0,69225.0,20925.0
2009-08-24,22247.0,307627.0,145331.0,31084.0,92196.0,NaN,9765.0,47371.0,58686.0,59279.0,...,94850.0,57075.0,256036.0,6499.0,45643.0,37588.0,261709.0,32212.0,66800.0,21776.0
2009-08-25,30891.0,246836.0,115840.0,18321.0,69281.0,NaN,8212.0,38613.0,51873.0,58042.0,...,70456.0,52132.0,211849.0,7968.0,65192.0,66748.0,228505.0,38307.0,57468.0,23235.0
2009-08-26,33334.0,286577.0,108570.0,20001.0,65988.0,NaN,11869.0,40547.0,40096.0,72002.0,...,60316.0,51363.0,187769.0,8462.0,40004.0,56715.0,158452.0,28738.0,46107.0,20250.0
2009-08-27,70176.0,265027.0,160421.0,22949.0,78030.0,NaN,16502.0,51713.0,46687.0,41460.0,...,58881.0,37824.0,213956.0,8673.0,40968.0,77898.0,304109.0,26442.0,19190.0,15259.0


# Задание 3

<img src="img/21_3.png" width=800>

In [13]:
import dask
from collections import defaultdict
from typing import Sized
import string
import heapq

In [14]:
def make_groups() -> dict:
    with open('data/task_3/movies.txt') as f:
        punctuation = set(string.punctuation) - {'&'}
        groups = defaultdict(list)
        for line in f:
            words = [w for w in line.split() if w not in punctuation]
            if words:
                first, middle, last = words[0], words[1:-1], words[-1]
                groups['first'].append(first)
                if last != first:
                    groups['last'].append(last)
                if middle:
                    groups['middle'].extend(middle)
        return groups

In [15]:
@dask.delayed(pure=True, traverse=False)
def most_longest(lst: str) -> list[Sized]:
    return heapq.nlargest(2, lst, key=len)

In [19]:
groups = make_groups()
delayed = []
for key, value in groups.items():
    delayed.append(most_longest(value))

result = [i for res in dask.compute(delayed)[0] for i in res]
result

['Crazy/Beautiful',
 'Supercapitalist',
 'Extra-Terrestrial',
 'Out-of-Towners',
 "Philosopher's",
 'Extraordinary']